# <center> ***Anomaly-Driven Video Summarization***</center>

#### **Vishnu Priyan Sellam Shanmugavel**
#### **A20561323**
#### **CS512 - F24** 

<a id="im"></a>
# <center>IMPORTING LIBRARIES</center> 

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import os

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import roc_curve, auc, roc_auc_score


from IPython.display import clear_output
import warnings
# warnings.filterwarnings('ignore')

<a id="hp"></a>
# <center>HYPERPARAMETRERS AND DIRECTORIES</center>

In [36]:
train_dir = "dataset/UCF_Crime/Train"
test_dir = "dataset/UCF_Crime/Test"

SEED = 12
IMG_HEIGHT = 64
IMG_WIDTH = 64
BATCH_SIZE = 64
EPOCHS = 1
LR =  0.00003
NUM_CLASSES = 14
NUM_FRAMES = 16
CHANNELS = 3
CLASS_LABELS = ['Abuse','Arrest','Arson','Assault','Burglary','Explosion','Fighting',"Normal",'RoadAccidents','Robbery','Shooting','Shoplifting','Stealing','Vandalism']

<a id="data"></a>
# <center> DATA LOADING AND PRE-PROCESSING</center>

Creating two different generator for train and test data

In [4]:
preprocess_fun = tf.keras.applications.densenet.preprocess_input

train_datagen = ImageDataGenerator(horizontal_flip=True,
                                   width_shift_range=0.1,
                                   height_shift_range=0.05,
                                   rescale = 1./255,
                                   preprocessing_function=preprocess_fun
                                  )
test_datagen = ImageDataGenerator(rescale = 1./255,
                                  preprocessing_function=preprocess_fun
                                 )

In [5]:
train_generator = train_datagen.flow_from_directory(directory = train_dir,
                                                    target_size = (IMG_HEIGHT ,IMG_WIDTH),
                                                    batch_size = BATCH_SIZE,
                                                    shuffle  = True , 
                                                    color_mode = "rgb",
                                                    class_mode = "categorical",
                                                    seed = SEED
                                                   )
test_generator = test_datagen.flow_from_directory(directory = test_dir,
                                                   target_size = (IMG_HEIGHT ,IMG_WIDTH),
                                                    batch_size = BATCH_SIZE,
                                                    shuffle  = False , 
                                                    color_mode = "rgb",
                                                    class_mode = "categorical",
                                                    seed = SEED
                                                  )

Found 1266345 images belonging to 14 classes.
Found 111308 images belonging to 14 classes.


## Data distribution (count) among all the classes

### Train data Distribution

In [6]:
fig = px.bar(x = CLASS_LABELS,
             y = [list(train_generator.classes).count(i) for i in np.unique(train_generator.classes)] , 
             color = np.unique(train_generator.classes) ,
             color_continuous_scale="Emrld") 
fig.update_xaxes(title="Classes")
fig.update_yaxes(title = "Number of Images")
fig.update_layout(showlegend = True,
    title = {
        'text': 'Train Data Distribution ',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

### Test Data Distribution

In [7]:
fig = px.bar(x = CLASS_LABELS,
             y = [list(test_generator.classes).count(i) for i in np.unique(test_generator.classes)] , 
             color = np.unique(train_generator.classes) ,
             color_continuous_scale="Emrld") 
fig.update_xaxes(title="Classes")
fig.update_yaxes(title = "Number of Images")
fig.update_layout(showlegend = True,
    title = {
        'text': 'Test Data Distribution ',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

### C3D and 3D ResNet50 Transfer Learning

In [39]:

# C3D feature extractor
def c3d_feature_extractor(inputs):
    x = tf.keras.layers.Conv3D(filters=64, kernel_size=(3, 3, 3), activation='relu')(inputs)
    x = tf.keras.layers.MaxPooling3D(pool_size=(1, 2, 2))(x)
    x = tf.keras.layers.Conv3D(filters=128, kernel_size=(3, 3, 3), activation='relu')(x)
    x = tf.keras.layers.MaxPooling3D(pool_size=(1, 2, 2))(x)
    x = tf.keras.layers.Conv3D(filters=256, kernel_size=(3, 3, 3), activation='relu')(x)
    x = tf.keras.layers.MaxPooling3D(pool_size=(1, 2, 2))(x)
    x = tf.keras.layers.Conv3D(filters=512, kernel_size=(3, 3, 3), activation='relu')(x)
    x = tf.keras.layers.MaxPooling3D(pool_size=(1, 2, 2))(x)
    x = tf.keras.layers.Flatten()(x)
    return x

# ResNet feature extractor
def resnet3d_feature_extractor(inputs):
    base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, CHANNELS))
    
    x = base_model(inputs)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(1024, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    
    return x

# Combined feature extractor
def combined_feature_extractor(inputs):
    # C3D features
    c3d_features = c3d_feature_extractor(inputs)
    
    resnet_input = tf.keras.layers.Lambda(lambda x: x[:, 0, :, :, :])(inputs) 
    resnet3d_features = resnet3d_feature_extractor(resnet_input)
    
    # Concatenate the features from both models
    combined_features = tf.keras.layers.Concatenate()([c3d_features, resnet3d_features])
    
    return combined_features

# Classifier
def classifier(inputs):
    x = tf.keras.layers.Dense(256, activation="relu")(inputs)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(1024, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(NUM_CLASSES, activation="softmax")(x)
    
    return x

# Final model
def final_model(inputs):
    combined_features = combined_feature_extractor(inputs)
    classification_output = classifier(combined_features)
    
    return classification_output

# Define and compile model
def define_compile_model():
    inputs = tf.keras.layers.Input(shape=(NUM_FRAMES, IMG_HEIGHT, IMG_WIDTH, CHANNELS))
    classification_output = final_model(inputs)
    model = tf.keras.Model(inputs=inputs, outputs=classification_output)

    model.compile(optimizer=tf.keras.optimizers.SGD(), 
                  loss='categorical_crossentropy',
                  metrics=[tf.keras.metrics.AUC()])

    return model

model = define_compile_model()
model.summary()


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_34      │ (None, 16, 64,    │          0 │ -                 │
│ (InputLayer)        │ 64, 3)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_22 (Conv3D)  │ (None, 14, 62,    │      5,248 │ input_layer_34[0… │
│                     │ 62, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling3d_12    │ (None, 14, 31,    │          0 │ conv3d_22[0][0]   │
│ (MaxPooling3D)      │ 31, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_23 (Conv3D)  │ (None, 12, 29,    │    221,312 │ max_pooling3d_12… │
│                     │ 29, 128)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling3d_13    │ (None, 12, 14,    │          0 │ conv3d_23[0][0]   │
│ (MaxPooling3D)      │ 14, 128)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_24 (Conv3D)  │ (None, 10, 12,    │    884,992 │ max_pooling3d_13… │
│                     │ 12, 256)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 64, 64, 3) │          0 │ input_layer_34[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling3d_14    │ (None, 10, 6, 6,  │          0 │ conv3d_24[0][0]   │
│ (MaxPooling3D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ resnet50            │ (None, 2, 2,      │ 23,587,712 │ lambda[0][0]      │
│ (Functional)        │ 2048)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_25 (Conv3D)  │ (None, 8, 4, 4,   │  3,539,456 │ max_pooling3d_14… │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 2048)      │          0 │ resnet50[0][0]    │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling3d_15    │ (None, 8, 2, 2,   │          0 │ conv3d_25[0][0]   │
│ (MaxPooling3D)      │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_14 (Dense)    │ (None, 1024)      │  2,098,176 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_3 (Flatten) │ (None, 16384)     │          0 │ max_pooling3d_15… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_10          │ (None, 1024)      │          0 │ dense_14[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 17408)     │          0 │ flatten_3[0][0],  │
│ (Concatenate)       │                   │            │ dropout_10[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_15 (Dense)    │ (None, 256)       │  4,456,704 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_11          │ (None, 256)       │          0 │ dense_15[0][0]    │
│ (Dropout)           │                   │            │                 

 Total params: 35,071,118 (133.79 MB)

 Trainable params: 35,017,998 (133.58 MB)

 Non-trainable params: 53,120 (207.50 KB)

<a id="train"></a>
# <center> Training </center> 

In [44]:
history = model.fit(x = train_generator,validation_data=test_generator,epochs = EPOCHS)
# Train the model using the image generators
# history = model.fit(
#     train_generator,
#     validation_data=test_generator,
#     epochs=EPOCHS,
#     steps_per_epoch=train_generator.samples // BATCH_SIZE,
#     validation_steps=test_generator.samples // BATCH_SIZE,
#     verbose=1
# )

ValueError: Input 0 of layer "functional_5" is incompatible with the layer: expected shape=(None, 16, 64, 64, 3), found shape=(None, 64, 64, 3)

<a id="auc"></a>
# <center> Multiclass AUC Curve </center> 

In [ ]:
preds = model.predict(test_generator)
y_test = test_generator.classes
fig, c_ax = plt.subplots(1,1, figsize = (15,8))

def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    for (idx, c_label) in enumerate(CLASS_LABELS):
        fpr, tpr, thresholds = roc_curve(y_test[:,idx].astype(int), y_pred[:,idx])
        c_ax.plot(fpr, tpr,lw=2, label = '%s (AUC:%0.2f)'  % (c_label, auc(fpr, tpr)))
    c_ax.plot(fpr, fpr, 'black',linestyle='dashed', lw=4, label = 'Random Guessing')
    return roc_auc_score(y_test, y_pred, average=average)

print('ROC AUC score:', multiclass_roc_auc_score(y_test , preds  , average = "micro"))
plt.xlabel('FALSE POSITIVE RATE', fontsize=18)
plt.ylabel('TRUE POSITIVE RATE', fontsize=16)
plt.legend(fontsize = 11.5)
plt.show()